# 

title: Water Rights Restored to the Gila River

subtitle: The impacts of irrigation on vegetation health in the Gila
River Valley

author:

-   Elsa Culler

-   Nate Quarderer

date: last-modified

image: /img/earth-analytics/water-rights/lesson-water-rights.png

image-alt: “Dry river with dead plants turns into a stream with living
plants”

description: \|

In 2004, the Akimel O’‘otham and Tohono O’’odham tribes won a water
rights settlement in the US Supreme Court. Using satellite imagery, we
can see the effects of irrigation water on the local vegetation.

learning-goals:

-   Open raster or image data using code

-   Combine raster data and vector data to crop images to an area of
    interest

-   Summarize raster values with stastics

-   Analyze a time-series of raster images

params:

id: stars

site_name: Gila River Indian Community

event: water rights case

data_dir: gila-river

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r ndvi_da gdf

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [2]:
# Import libraries

import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt

import earthpy
import glob
import hvplot.pandas
import hvplot.xarray

In [3]:
# check out the dataarray
ndvi_da.head()

<xarray.Dataset> Size: 386B
Dimensions:      (date: 5, band: 1, y: 5, x: 5)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 40B 2001-01-14 2001-01-16 ... 2001-01-20
Data variables:
    NDVI         (date, band, y, x) int16 250B 8282 6146 3796 ... 5887 5887 5359

# STEP 3: Plot NDVI

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Plot the change in NDVI spatially</div></div><div class="callout-body-container callout-body"><p>Complete the following:</p>
<ul>
<li>Select data from 2021 to 2023 (3 years after the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="1"></span>)</li>
<li>Take the temporal mean (over the <strong>date</strong>, not
spatially)</li>
<li>Get the NDVI variable (should be a DataArray, not a Dataset)</li>
<li>Repeat for the data from 2018 to 2020 (3 years before the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="2"></span>)</li>
<li>Subtract the 2018-2020 time period <strong>from</strong> the
2021-2023 time period</li>
<li>Plot the result using a <strong>diverging</strong> color map like
<code>cmap=plt.cm.PiYG</code></li>
</ul>
<p>There are different types of color maps for different types of data.
In this case, we want decreases to be a different color from increases,
so we should use a <strong>diverging</strong> color map. Check out
available colormaps in the <a
href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">matplotlib
documentation</a>.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?</div></div><div class="callout-body-container callout-body"><p>For an extra challenge, add the <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary to the plot.</p></div></div>

In [4]:
# Convert the dates to datetime format (I used chatGPT to help figure this out)
ndvi_da['date'] = pd.to_datetime(ndvi_da['date'].values, format='%Y-%m-%d')

# Select data from 2021-2023
ndvi_aft = ndvi_da.sel(date=ndvi_da['date'].dt.year.isin([2021, 2022, 2023]))

# Calculate the temporal mean
ndvi_mean_aft = ndvi_aft.mean(dim='date')
#ndvi_mean_aft.head()

# Get the NDVI variable

ndvi_post = ndvi_mean_aft['NDVI']
ndvi_post.head()


<xarray.DataArray 'NDVI' (band: 1, y: 5, x: 5)> Size: 200B
array([[[6216.        , 4986.42857143, 4097.        , 4538.35714286,
         3148.07142857],
        [5205.42857143, 4797.71428571, 3546.42857143, 3032.42857143,
         3032.42857143],
        [3378.42857143, 3807.71428571, 4127.21428571, 3794.78571429,
         3794.78571429],
        [3776.5       , 4303.92857143, 4283.71428571, 4283.71428571,
         4132.64285714],
        [2985.28571429, 3865.5       , 3959.78571429, 3959.78571429,
         3624.78571429]]])
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0

In [5]:
# Select data from 2021-2023
ndvi_bef = ndvi_da.sel(date=ndvi_da['date'].dt.year.isin([2018, 2019, 2020]))

# Calculate the temporal mean
ndvi_mean_bef = ndvi_bef.mean(dim='date')
#ndvi_mean_bef.head()

# Get the NDVI variable

ndvi_pre = ndvi_mean_bef['NDVI']
ndvi_pre.head()


<xarray.DataArray 'NDVI' (band: 1, y: 5, x: 5)> Size: 200B
array([[[4999.28571429, 4429.42857143, 4491.28571429, 5376.38095238,
         6191.80952381],
        [4529.61904762, 4100.42857143, 3608.        , 3584.52380952,
         3584.52380952],
        [3453.61904762, 3865.66666667, 4268.76190476, 4112.71428571,
         4112.71428571],
        [3961.28571429, 4280.71428571, 4594.23809524, 4594.23809524,
         4626.95238095],
        [3695.19047619, 4161.42857143, 4516.71428571, 4516.71428571,
         4214.14285714]]])
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0

In [6]:
# Subtract before from after
ndvi_dif = ndvi_post - ndvi_pre
ndvi_dif.head()

<xarray.DataArray 'NDVI' (band: 1, y: 5, x: 5)> Size: 200B
array([[[ 1216.71428571,   557.        ,  -394.28571429,  -838.02380952,
         -3043.73809524],
        [  675.80952381,   697.28571429,   -61.57142857,  -552.0952381 ,
          -552.0952381 ],
        [  -75.19047619,   -57.95238095,  -141.54761905,  -317.92857143,
          -317.92857143],
        [ -184.78571429,    23.21428571,  -310.52380952,  -310.52380952,
          -494.30952381],
        [ -709.9047619 ,  -295.92857143,  -556.92857143,  -556.92857143,
          -589.35714286]]])
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0

In [7]:
# Select just Band 1
ndvi_dif_squeeze = ndvi_dif.sel(band=1)

# Plot the difference
(
    ndvi_dif_squeeze.hvplot(x='x', y='y', cmap=plt.cm.PiYG, geo=True, label='')
    *
    gdf.hvplot(geo=True, fill_color=None, line_color='black')
)

:Overlay
   .Image.I    :Image   [x,y]   (NDVI)
   .Polygons.I :Polygons   [Longitude,Latitude]

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [8]:
%store ndvi_da ndvi_dif_squeeze gdf

Stored 'ndvi_da' (Dataset)
Stored 'ndvi_dif_squeeze' (DataArray)
Stored 'gdf' (GeoDataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!